In [17]:
%pip install tensorflow-gpu==1.15.5
%pip install tqdm
%pip install tffm
%pip install wandb
%pip install scipy

## EDIT /usr/local/lib/python3.7/dist-packages/tffm base.py 
## AND **RESTART RUNTIME**

# 1

In [1]:
import numpy as np
import tensorflow as tf
import time
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import tffm

# 2

In [4]:
train90_file =      "/content/drive/MyDrive/HOFM/CIL_data90.train"
train90aug_file = "/home/ico/PycharmProjects/CIL-2021/baselines/HighOrderFM/CIL_data90aug.train.libfm"
train100_file =     "/content/drive/MyDrive/HOFM/CIL_data100.train"
train100aug_file = "/home/ico/PycharmProjects/CIL-2021/baselines/HighOrderFM/CIL_data100aug.train.libfm"
valid_file =        "/content/drive/MyDrive/HOFM/CIL_data10.valid"
validaug_file = "/home/ico/PycharmProjects/CIL-2021/baselines/HighOrderFM/CIL_dataaug.valid.libfm"
sub_file =          "/content/drive/MyDrive/HOFM/CIL_data.submission"
subaug_file = "/home/ico/PycharmProjects/CIL-2021/baselines/HighOrderFM/CIL_dataaug.submission.libfm"

user_bagofitems_dict = dict()
item_bagofusers_dict = dict()

for in_file in [train100_file, sub_file]:
    with open(in_file) as inf:
        for in_line in inf.readlines():
            split_line = in_line.split("\t")
            user = split_line[0]
            item = split_line[1]
            if user not in user_bagofitems_dict:
                # new user: create new set of items rated by the user
                user_bagofitems_dict[user] = set()
            user_bagofitems_dict[user].add(item)
            if item not in item_bagofusers_dict:
                # new item: create new set of users that rated the item
                item_bagofusers_dict[item] = set()
            item_bagofusers_dict[item].add(user)


# 3

In [5]:
for user in user_bagofitems_dict.keys():
    user_bagofitems_dict[user] = sorted(user_bagofitems_dict[user])
for item in item_bagofusers_dict.keys():
    item_bagofusers_dict[item] = sorted(item_bagofusers_dict[item])

# 4

In [6]:
from scipy.sparse import coo_matrix, lil_matrix
file_idx = 1
nrows = [1054805, 1176952, 122147, 1176952]
nelems = [4438418149, 4954866100, 516447951, 4955322616]
for in_file in [train90_file, train100_file, valid_file, sub_file]:
    print("Processing file", file_idx)
    
    line_num = 0
    idx = 0
    X = lil_matrix((nrows[file_idx-1],11000))
    with open(in_file) as inf:
#         row = np.ndarray(nelems[file_idx - 1])
#         col = np.ndarray(nelems[file_idx - 1])
#         data = np.ndarray(nelems[file_idx - 1])
        y = np.ndarray(nrows[file_idx - 1])
        for in_line in inf.readlines():
            split_line = in_line.split("\t")
            user = int(split_line[0]) - 1
            item = int(split_line[1]) - 1
            rating = float(split_line[2][:-1])

            y[line_num] = rating

            X[line_num,user] = 1

            X[line_num,10000+item] = 1

            line_num += 1
            if line_num % 100000 == 0:
                print(line_num)
    
    if file_idx == 1:
        train90_X = X
        train90_y = y
    elif file_idx == 2:
        train100_X = X
        train100_y = y
    elif file_idx == 3:
        val10_X = X
        val10_y = y
    elif file_idx == 4:
        sub_X = X
    
    file_idx += 1
    
    

Processing file 1
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
Processing file 2
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
Processing file 3
100000
Processing file 4
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000


# Regression example

# 5

In [7]:


# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
def train():
    from tensorflow.python.client import device_lib

    def get_available_gpus():
        local_device_protos = device_lib.list_local_devices()
        return [x.name for x in local_device_protos if x.device_type == 'GPU']

    print(get_available_gpus())


    import tensorflow as tf
    from tffm import TFFMRegressor
    from sklearn.metrics import mean_squared_error

    with wandb.init() as run:
        config = wandb.config
        with tf.Session() as sess:
            with tf.device('/device:GPU:0'):
                order = 3

                model = TFFMRegressor(
                    order=3, 
                    rank=16, 
                    optimizer=tf.train.AdamOptimizer(learning_rate=0.008674), 
                    # optimizer=tf.train.AdamOptimizer(learning_rate=config.lr),
                    n_epochs=10, 
                    # batch_size=config.batch_size,
                    batch_size=27242,
                    # init_std=config.init_std,
                    init_std=0.04752,
                    # reg=config.reg,
                    reg = 0.04616,
                    seed=config.seed,
                    verbose=1
                )

                model.fit(train90_X.astype(float), train90_y.astype(float), show_progress=True)
                predictions = model.predict(val10_X.astype(float))
                predictions = predictions.clip(1.0,5.0)
                #     print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions)))
                rmse = mean_squared_error(val10_y.astype(float), predictions, squared=False)
                print('[order={}] rmse: {}'.format(order, rmse))
                wandb.log({"loss": rmse})


In [14]:
# SAVE THE MODEL
# Add ops to save and restore all the variables.
# Save the variables to disk.
save_path = '/content/drive/MyDrive/HOFM/state15epochs.tf'
model.save_state(save_path)
print("Model saved in path: %s" % save_path)
# with tf.Session() as sess:
#   with tf.device('/device:GPU:0'):
#     print("Model weights:",model.weights)
    # savetxt('data.csv', model.weights, delimiter=',')

predictions = predictions.clip(1.0,5.0)
#     print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions)))
rmse = mean_squared_error(val10_y.astype(float), predictions, squared=False)
print('[order={}] rmse: {}'.format(order, rmse))

Model saved in path: /content/drive/MyDrive/HOFM/state15epochs.tf
[order=3] rmse: 0.9924351095409815


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="CIL-2021")

## Sweep find batch_size, init_std, lr, reg

In [ ]:
%%wandb 

sweep_config = {
  "name" : "HOFM-sweep",
  "method" : "bayes",
  "metric": {
    "name": "loss",
    "goal": "minimize"
  },
  "parameters" : {
    "batch_size" : {
      "distribution": "int_uniform",
      "max":32768,
      "min":16
    },
    "init_std" :{
      "distribution": "uniform",
      "min":0.01,
      "max": 0.2
    },
    "lr" :{
      "distribution": "log_uniform",
      "min": -5,
      "max": -2
    },
    "reg" :{
      "distribution": "uniform",
      "min": 0.01,
      "max": 0.1
    }
  }
}


# sweep_id = wandb.sweep()
wandb.agent("fgossi/uncategorized/yemg0upk", function=train)

## Run with custom parameters

In [8]:
from types import SimpleNamespace
config = SimpleNamespace(batch_size=27242,init_std=0.04752,lr=0.008674,reg=0.04616)
print(config.batch_size, config.init_std, config.lr, config.reg)

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())


import tensorflow as tf
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error

with tf.Session() as sess:
  with tf.device('/device:GPU:0'):
      order = 3

      model = TFFMRegressor(
          order=3, 
          rank=10, 
      #     optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), 
          optimizer=tf.train.AdamOptimizer(learning_rate=config.lr),
          n_epochs=1, 
          batch_size=config.batch_size,
          init_std=config.init_std,
          reg=config.reg,
          seed=6867,
          verbose=2
      )

      # LOAD MODEL
      # internally model need to allocate memory before load previous weights,
      # so need to set num_features explicitly
      model.core.set_num_features(train90_X.shape[1])
      model.load_state('/content/drive/MyDrive/HOFM/state15epochs.tf')
      # print("MODEL W:",model.weights)
      P = model.weights

      # model.fit(train100_X.astype(float), train100_y.astype(float), show_progress=True)
      predictions = model.predict(val10_X.astype(float))
      predictions = predictions.clip(1.0,5.0)
      #     print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions)))
      rmse = mean_squared_error(val10_y.astype(float), predictions, squared=False)
      print('[order={}] rmse: {}'.format(order, rmse))
      wandb.log({"loss": rmse})




27242 0.04752 0.008674 0.04616
['/device:GPU:0']




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/HOFM/state15epochs.tf
MODEL W: [array([[0.34893435],
       [0.45540506],
       [0.37341544],
       ...,
       [0.4574281 ],
       [0.46684876],
       [0.39720348]], dtype=float32), array([[ 0.4579176 , -0.65613383, -0.31547466, ..., -0.51895255,
        -0.5475446 , -0.372314  ],
       [ 0.41873342, -0.499632  , -0.59390277, ..., -0.26028243,
        -0.53897935, -0.45188987],
       [ 0.26604146, -0.6121737 , -0.46982232, ..., -0.35925657,
        -0.45513788, -0.33281693],
       ...,
       [ 0.5522373 , -0.4767486 , -0.30707687, ..., -0.3226085 ,
        -0.634228  , -0.38051546],
       [ 0.41107842, -0.6041327 ,  0.10575255, ..., -0.61495334,
        -0.39255616, -0.45185563],
       [ 0.44170204, -0.71614724, -0.39578992, ..., -0.46579742,
        -0.38248876, 

NameError: ignored

In [30]:
# save to csv file
from numpy import savetxt
savetxt('/content/drive/MyDrive/HOFM/predsEp15.csv', predictions, delimiter=',')

## SWEEP FIND random seed

In [ ]:
%%wandb 

sweep_config = {
  "name" : "HOFM-sweep",
  "method" : "random",
  "metric": {
    "name": "loss",
    "goal": "minimize"
  },
  "parameters" : {
    "seed" : {
      "distribution": "int_uniform",
      "max":10000,
      "min":1
    }
  }
}


sweep_id = wandb.sweep(sweep_config)
wandb.agent(sweep_id, function=train)
# when sweep is already present:
# wandb.agent("fgossi/uncategorized/ap71v6fx", function=train)


## ONLY ORDER 3 predictions

In [13]:
# P[2]
def dot(q1, q2, q3):
  sum = 0
  for i in range(len(q1)):
    sum += q1[i]*q2[i]*q3[i]
  return sum

Q = P[2]
X = val10_X
val10_X.shape
pred3ord = np.zeros_like(val10_y)
for row, x in tqdm(enumerate(val10_X)):
  for j1 in tqdm(range(11000)):
    for j2 in tqdm(range(j1+1, 11000, 1)):
      for j3 in range(j2+1, 11000, 1):
        pred3ord[row] = dot(Q[j1],Q[j2],Q[j3])*X[row,j1]*X[row,j2]*X[row,j3]

pred3ord = pred3ord.clip(1.0,5.0)
rmse = mean_squared_error(val10_y.astype(float), pred3ord, squared=False)
print(rmse)

Streaming output truncated to the last 5000 lines.




  3%|▎         | 309/10999 [01:12<44:15,  4.03it/s]




  3%|▎         | 310/10999 [01:12<45:00,  3.96it/s]




  3%|▎         | 311/10999 [01:12<44:33,  4.00it/s]




  3%|▎         | 312/10999 [01:13<44:01,  4.05it/s]




  3%|▎         | 313/10999 [01:13<43:49,  4.06it/s]




  3%|▎         | 314/10999 [01:13<43:50,  4.06it/s]




  3%|▎         | 315/10999 [01:13<43:32,  4.09it/s]




  3%|▎         | 316/10999 [01:14<43:58,  4.05it/s]




  3%|▎         | 317/10999 [01:14<44:04,  4.04it/s]




  3%|▎         | 318/10999 [01:14<43:48,  4.06it/s]




  3%|▎         | 319/10999 [01:14<43:39,  4.08it/s]




  3%|▎         | 320/10999 [01:15<43:30,  4.09it/s]




  3%|▎         | 321/10999 [01:15<43:10,  4.12it/s]




  3%|▎         | 322/10999 [01:15<43:19,  4.11it/s]




  3%|▎         | 323/10999 [01:15<43:36,  4.08it/s]




  3%|▎         | 324/10999 [01:16<43:50,  4.06it/s]




  3%|▎         | 325/10999 [01:16<43:49,  4.06it/

KeyboardInterrupt: ignored